In [ ]:
from __future__ import print_function

from intervaltree import IntervalTree, Interval
from collections import defaultdict

In [ ]:
def overlap_intervals(self, other):
    splits = (self | other)
    splits.split_overlaps()
    self_int_other = IntervalTree(filter(lambda r: self.overlaps(r) and other.overlaps(r), splits))
    return self_int_other


def overlap_all(trees):
    if len(trees) in (0, 1):
        return None

    overlap = overlap_intervals(trees[0], trees[1])
    for t in trees[2:]:
        overlap = overlap_intervals(overlap, t)
    overlap.merge_overlaps(data_reducer)

    return overlap


def data_reducer(current_data, new_data):
    return current_data | new_data

In [ ]:
gtrees = defaultdict(dict)
all_genes = set()
files = defaultdict(lambda: defaultdict(list))
#allref= defaultdict(lambda: defaultdict(list))

In [ ]:
for f in ('vqc', 'iqc', 'sqc', 'mqc'):
    for line in open(f): 
            x = line.rstrip().split(' ')
            if  "Real" in  str(x[0]):
                files[f][x[14]].append((int(x[3]), int(x[4])))
print('done')

In [14]:
gtrees = defaultdict(dict)
atrees = defaultdict(dict)
all_genes = set()
for f in files:
    for g in files[f]:
        all_genes.add(g)
        gtrees[g][f] = IntervalTree(Interval(iv[0], iv[1], set([f]))
                                    for iv in files[f][g])
        gtrees[g][f].chop(0,1000000)
        #gtrees[g][f].split_overlaps()
print('done')

done


In [15]:
cutoff =10
out =open('unaligned-regions','w+')
overlap_gtrees = {}
for gene in all_genes:
    overlap_gtrees[gene] = overlap_all(list(gtrees[gene].values()))
    
    if overlap_gtrees[gene]:
        print('@',gene,":",overlap_gtrees[gene].begin( ),",",overlap_gtrees[gene].end(), len(overlap_gtrees[gene])) 
        if len(overlap_gtrees[gene]) >cutoff:
                for i in overlap_gtrees[gene]:
                        print(gene,',',i,',',file=out)
print ('done ')                


@ Clostridium_thermocellum_ATCC_27405 : 2003160 , 2010929 2
@ Caldisaccharolyticus_DSM_8903 : 520488 , 1609657 6
@ Geobacter_sulfurreducens_PCA : 2015315 , 3543539 7
@ Enterococcus_faecalis_V583 : 17366 , 3218031 70
@ Burkholderia1_xenovorans_LB400_chromosome_1__complete_sequence : 168126 , 784099 3
@ Chlorobiumphaeovibrioides_DSM_265 : 1929914 , 1930267 1
@ Sulfurihydrogenibium_yellowstonense_SS-5 : 883 , 1524768 22
@ Caldibescii_DSM_6725 : 2803730 , 2803948 1
@ Pyrococcus_furiosus_DSM_3638 : 275668 , 1903011 11
@ Akkermansia_muciniphila_ATCC_BAA-835 : 512001 , 2610978 7
@ Nostoc_sp._PCC_7120_DNA : 1273737 , 1292767 2
@ Herpetosiphon_aurantiacus_ATCC_23779 : 4123453 , 4123507 1
@ Bacteroides_thetaiotaomicron_VPI-5482 : 6000673 , 6060083 1
@ Deinococcus_radiodurans_R1_chromosome_1__complete_sequence : 1 , 3060986 3
@ Sulfolobus_tokodaii_str._7_DNA__complete_genome : 2240715 , 2269526 1
@ Porphyromonas_gingivalis_ATCC_33277_DNA : 1449250 , 1492705 2
@ Bordetella_bronchiseptica_strain_RB

In [16]:
#parse case files
gdict=defaultdict(list)
input=open('unaligned-regions','r')
for line in input:
        line=line.split(',')
        #print(line)
        gene=line[0]
        x=line[1].split('(')
        x=x[1]
        y=line[2]
        #print(x,"-",y)
        gdict[line[0]].append(x)
        gdict[line[0]].append(y)


for g in gdict.keys(): 
         i=1
         l=len(gdict[g])
         min =20700000
         max =-1
         sum=0
         while(i <l-1) :
                gap=abs( int(gdict[g][i+1]) -int(gdict[g][i]) )
                sum+=gap
                if min >gap: 
                        min=gap
                if max <gap:
                        max=gap
                i=i+2
         print(g,"has ",l/2,"unaligned with min/max/average gaps:",str(min), "/",str(max),"/",(sum/l))

Enterococcus_faecalis_V583  has  70 unaligned with min/max/average gaps: 26 / 2759637 / 548867
Desulfovibrio_vulgaris_DP4  has  26 unaligned with min/max/average gaps: 84280 / 3216841 / 875212
Sulfurihydrogenibium_yellowstonense_SS-5  has  22 unaligned with min/max/average gaps: 7448 / 1481175 / 229042
Thermus_thermophilus_HB27  has  48 unaligned with min/max/average gaps: 8222 / 1327689 / 300080
Pyrococcus_furiosus_DSM_3638  has  11 unaligned with min/max/average gaps: 73043 / 1407633 / 284824


In [ ]:
for i in overlap_gtrees['Desulfovibrio_vulgaris_DP4']:
      print(i)

In [22]:
#t = IntervalTree([Interval(2, 5),Interval(7,9)])
t=IntervalTree([Interval(0,10)])
t.chop(3,7)
#t.split_overlaps()
sorted(t)

[Interval(0, 3), Interval(7, 10)]